In [83]:
# packages
import os
import shutil
import csv
import re
import pandas as pd
from Bio import SeqIO

In [84]:
# classes and functions
class SeqContainer:
    general_mod = []
    accession = ''
    ph_init_seq = ''
    ph_trans_seq = ''
    db_seq = ''
    ph_pep_aa = []
    ph_pep_pos = []
    ph_prot_pos = []

# fills inital SeqContainer datastructure
# returns list of SeqContainer [0] and list of modification [1]
def buildSeqContainer(db_path, analysis_path):
    # read input 
    df = pd.read_csv(analysis_path)
    seq_col = df.sequence
    accession_col = df.accessions

    if(len(seq_col) != len(accession_col)):
        print("Error: sequence and accessions columns are not equal in lenght.")
        sys.exit()

    # find all modifictions apart from Phosphorylation ()
    l_mod = []
    for entry in seq_col:
        for m in re.findall('\(.+?\)', entry):
            l_mod.append(m)
    l_mod = list(set(l_mod))

    # remove 'Phospho' modification will be used later on.
    l_mod.remove('(Phospho)')

    # dictionary accession phosphosites
    d_ac_ph = {}
    for i in range(len(seq_col)):
        if (seq_col[i] != "UNIDENTIFIED PEPTIDE"):
            d_ac_ph[accession_col[i]] = seq_col[i]

    # dictionary accession sequence (fasta)
    d_id_seq = {}
    for seq_record in SeqIO.parse(db_path, "fasta"):
        d_id_seq[seq_record.id] = seq_record.seq

    # combine the two dict in class SeqContainer 
    l_seqc = [] 
    l_seqc_mod = []
    for k_ph, v_ph in d_ac_ph.iteritems():
        for k_db, v_db in d_id_seq.iteritems():
            if (k_ph == k_db):
                seqc = SeqContainer()
                seqc.general_mod = l_mod
                seqc.accession = k_ph
                seqc.ph_init_seq = v_ph
                seqc.db_seq = v_db
                l_seqc.append(seqc)  
                
    return l_seqc
    
# extract phosphosite position and aminoacid in peptide and protein
# returns updated SeqContainer
def extractPhosphoPosition(SeqContainer):
    ph_str = SeqContainer.ph_init_seq
    if (ph_str == ""):
        print("Warning: Empty peptide squence was provided")
    ph_str = ph_str.replace('.','')
    # remove all modification apart from 'Phospho'
    for element in SeqContainer.general_mod:
        if element in ph_str:
            ph_str = ph_str.replace(element,'')
    ph_str_p = ph_str
    ph_str_a = ph_str_p.replace('(Phospho)','')
    SeqContainer.ph_trans_seq = ph_str_a

    l_site = []
    for m in re.finditer('(Phospho)', ph_str_p):
        l_site.append(m.start())

    # aminoacid = elem-2
    # position = elem-1
    l_pep_aa = []
    l_pep_pos = []
    for elem in l_site:
        l_pep_aa.append(ph_str_p[elem-2])
        l_pep_pos.append(elem-1)

    # correct for the lenght of "(Phospho)"
    for i in range(len(l_pep_pos)):
        l_pep_pos[i] = l_pep_pos[i] - (i*9)

    SeqContainer.ph_pep_aa = l_pep_aa
    SeqContainer.ph_pep_pos = l_pep_pos

    # if position was located find position in protein
    if (len(SeqContainer.ph_pep_aa) != 0 and len(SeqContainer.ph_pep_pos) != 0):
        peptide_position = []
        if (len(SeqContainer.db_seq) != 0):
        # look for position of peptide in db_sequence 
            for m in re.finditer(str(SeqContainer.ph_trans_seq), str(SeqContainer.db_seq)):
                peptide_position.append(m.start())
        else: 
            print("Database sequence was not found")

        if (len(peptide_position) != 0):
            l_prot_pos = []
            # add the posotion to the value in the peptide
            for element in SeqContainer.ph_pep_pos:
                l_prot_pos.append(element + peptide_position[0])
            SeqContainer.ph_prot_pos = l_prot_pos
        else:
            print("Was not able to localise peptide in database seqence")
    #else:
    #    print("The peptide sequence " + SeqContainer.ph_init_seq + " did not contain any phosphosites.")
    
    return SeqContainer

# build new dataframe
def buildDataFrameFromContainer(l_container):
    # get number of columns which are need in the csv
    max_len_position = 0
    for entry in l_container:
        len_position = len(entry.ph_pep_pos)
        if len_position > max_len_position:
            max_len_position = len_position

    # columns pep_position, prot_position
    # saved as AAPos S5 Y12 - S143 Y 150 
    # max_len_position * 2 + 2 (asccession, pep_seq)
    header_pep = []
    header_prot = []
    for i in range(max_len_position):
        header_pep.append('position_pep_'+str(i))
        header_prot.append('position_prot_'+str(i))

    header = [['accessions','sequence']]
    header.append(header_pep)
    header.append(header_prot)

    # flatten header list 
    header = [y for x in header for y in x]

    # make new dataframe
    df_container = pd.DataFrame(columns = header)

    # df[row,col]
    i=0
    for element in l_container:
        df_container.at[i,'accessions'] = element.accession
        df_container.at[i,'sequence'] = element.ph_init_seq
        if (len(element.ph_pep_aa) > 0):
            for j in range(len(element.ph_pep_aa)):
                df_container.at[i,'position_pep_'+str(j)] = str(str(element.ph_pep_aa[j])+str(element.ph_pep_pos[j]))
                df_container.at[i,'position_prot_'+str(j)] = str(str(element.ph_pep_aa[j])+str(element.ph_prot_pos[j]))
        i+=1

    return df_container

In [85]:
def main():
    print("Starting the extraction")

    # input paths 
    db_path = "/Volumes/elements/Ph_analysis/database/database/2018/swissprot_human_crap_decoy_20181017.fasta" 
    analysis_path = "/Volumes/elements/Ph_analysis/results/5min_HILIC/ConsensusMapNormalizer/ConsensusMapNormalizer.csv"
    output_path = "/Volumes/elements/Ph_analysis/results/5min_HILIC/ConsensusMapNormalizer/ConsensusMapNormalizer_output.csv"

    # TODO
    # check if paths are empty
    # check if analysis path is the same as output path 

    # build the inital SeqContainer datastructure
    l_container = buildSeqContainer(db_path, analysis_path)

    # extract phosphosite position and aminoacid in peptide and protein
    for entry in l_container:
        extractPhosphoPosition(entry)

    # build new dataframe from data in the container
    df_new = buildDataFrameFromContainer(l_container)

    # analysis_path dataframe
    df_in = pd.read_csv(analysis_path)

    # merge the two dataframes by accessions and sequence -> generate final output
    df_out = pd.merge(df_in, df_new,  how='left', left_on=['accessions','sequence'], right_on = ['accessions','sequence'])

    # write output
    df_out.to_csv(output_path, sep=',', encoding='utf-8')
    
    print("Done")
    
    return 0

In [86]:
if __name__ == "__main__":
    # execute only if run as a script
    main()

Starting the extraction
Done
